Выгружаем данные

In [1]:
import pandas as pd
import folium
import re
import json
from folium.plugins import HeatMap, MarkerCluster


In [2]:
# Загрузка данных - используем первую строку как заголовки (английские названия)
privatization = pd.read_csv("privatization.csv", sep=';', skiprows=1, low_memory=False)
rent = pd.read_csv("rent.csv", sep=';', skiprows=1, low_memory=False)
land_plot = pd.read_csv("land_plot.csv", sep=';', skiprows=1, low_memory=False)
premises = pd.read_csv("premises.csv", sep=';', skiprows=1, low_memory=False)

# Пропускаем строку с русскими названиями
privatization = privatization.iloc[1:].reset_index(drop=True)
rent = rent.iloc[1:].reset_index(drop=True)
land_plot = land_plot.iloc[1:].reset_index(drop=True)
premises = premises.iloc[1:].reset_index(drop=True)

Создаем базовую карту

In [3]:
# Создаем карту и FeatureGroup для каждого слоя
m = folium.Map(location=[55.75, 37.61], zoom_start=10)

# Группы для каждого слоя
privatization_group = folium.FeatureGroup(name='Объекты приватизации', show=False)
rent_group = folium.FeatureGroup(name='Объекты аренды', show=False)
premises_group = folium.FeatureGroup(name='Нежилые помещения', show=False)
heatmap_group = folium.FeatureGroup(name='Плотность нежилых помещений', show=True)
land_group = folium.FeatureGroup(name='Земельные участки', show=False)

Создаем 1 слой (объекты приватизации)

In [4]:
try:
    privatization_cluster = MarkerCluster().add_to(privatization_group)
    added_count = 0
    
    for idx, row in privatization.iterrows():
        coords = None
        
        # Проверяем столбец "Координаты объекта" (формат "lat,lon")
        if 'Координаты объекта' in row and pd.notna(row['Координаты объекта']) and str(row['Координаты объекта']).strip():
            try:
                coords_str = str(row['Координаты объекта'])
                if ',' in coords_str:
                    parts = coords_str.split(',')
                    lat = float(parts[0].strip())
                    lon = float(parts[1].strip())
                    # Проверяем, что координаты в пределах Москвы
                    if 54 < lat < 57 and 36 < lon < 39:
                        coords = [lat, lon]
            except:
                pass
        
        # Если не нашли в "Координаты объекта", проверяем geoData
        if not coords and 'geoData' in row and pd.notna(row['geoData']) and str(row['geoData']).strip():
            try:
                geo_str = str(row['geoData'])
                if '{' in geo_str and 'coordinates' in geo_str:
                    # Чистим строку для парсинга JSON
                    geo_str_clean = geo_str.replace("coordinates=", '"coordinates":').replace("type=", '"type":')
                    geo = json.loads(geo_str_clean)
                    if geo['type'] == 'Point':
                        # В GeoJSON формате: [lon, lat]
                        lon = geo['coordinates'][0]
                        lat = geo['coordinates'][1]
                        if 54 < lat < 57 and 36 < lon < 39:
                            coords = [lat, lon]
            except:
                pass
        
        if coords:
            added_count += 1
            folium.Marker(
                coords,
                popup=f"""<b>Адрес:</b> {row.get('Адрес', '')}<br>
                       <b>Тип:</b> {row.get('Тип объекта', '')}<br>
                       <b>Площадь:</b> {row.get('Площадь (кв. м)', '')} кв.м<br>
                       <b>Стартовая цена:</b> {row.get('Стартовая цена (руб.)', '')} руб""",
                icon=folium.Icon(icon='home', color='green')
            ).add_to(privatization_cluster)
    
    # Добавляем слой на карту
    privatization_group.add_to(m)
    print(f"Добавлено объектов приватизации: {added_count} из {len(privatization)}")
except Exception as e:
    print(f"Ошибка слоя приватизации: {e}")

Добавлено объектов приватизации: 85115 из 135645


Создаем 2 слой (объекты аренды)

In [5]:
try:
    rent_cluster = MarkerCluster().add_to(rent_group)
    added_count = 0
    
    for idx, row in rent.iterrows():
        coords = None
        
        # Проверяем geoData (основной источник координат)
        if 'geoData' in row and pd.notna(row['geoData']) and str(row['geoData']).strip():
            try:
                geo_str = str(row['geoData'])
                # Используем регулярное выражение для извлечения координат
                match = re.search(r'coordinates=\[([0-9.]+),\s*([0-9.]+)\]', geo_str)
                if match:
                    lon = float(match.group(1))
                    lat = float(match.group(2))
                    # Проверяем, что координаты в пределах Московского региона
                    if 54 < lat < 57 and 36 < lon < 39:
                        coords = [lat, lon]
            except Exception as e:
                continue
        
        if coords:
            added_count += 1
            # Получаем площадь для размера маркера
            try:
                area = float(str(row.get('Площадь (кв. м)', '100')).replace(',', '.'))
                radius = min(max(area / 50, 5), 20)  # Радиус от 5 до 20
            except:
                radius = 8
            
            # Получаем арендную плату
            try:
                rent_price = float(str(row.get('Начальный размер годовой арендной платы (руб.)', '0')).replace(',', '.'))
                rent_formatted = f"{rent_price:,.0f}".replace(',', ' ')
            except:
                rent_formatted = "не указана"
            
            folium.CircleMarker(
                coords,
                radius=radius,
                popup=f"""<b>Адрес:</b> {row.get('Адрес', '')}<br>
                       <b>Площадь:</b> {row.get('Площадь (кв. м)', '')} кв.м<br>
                       <b>Аренда:</b> {rent_formatted} руб/год<br>
                       <b>Назначение:</b> {row.get('Назначение использования объекта нежилого фонда', 'не указано')}""",
                color='blue',
                fill=True,
                fillColor='lightblue',
                fillOpacity=0.7
            ).add_to(rent_cluster)
    
    # Добавляем слой на карту
    rent_group.add_to(m)
    print(f"Добавлено объектов аренды: {added_count} из {len(rent)}")
except Exception as e:
    print(f"Ошибка слоя аренды: {e}")

Добавлено объектов аренды: 2626 из 2626


Создаем 3 слой (нежилые помещения)

In [6]:
try:
    # 1. Данные уже загружены в ячейке выше
    
    # 2. Проверка и подготовка данных
    required_cols = ['Адрес', 'Площадь (кв.м.)', 'geoData']
    for col in required_cols:
        if col not in premises.columns:
            raise ValueError(f"Отсутствует обязательный столбец: {col}")
    
    # Преобразуем площадь в числовой формат (уже числовой, просто копируем)
    premises['Area'] = pd.to_numeric(premises['Площадь (кв.м.)'], errors='coerce')
    
    # 3. Улучшенная функция извлечения координат
    def extract_coords_from_csv(geo_str):
        try:
            if pd.isna(geo_str) or not isinstance(geo_str, str):
                return None
                
            # Используем регулярное выражение для извлечения координат
            match = re.search(r'coordinates=\[([0-9.]+),\s*([0-9.]+)\]', str(geo_str))
            if match:
                lon = float(match.group(1))
                lat = float(match.group(2))
                # Проверяем, что координаты в пределах Московского региона
                if 54 < lat < 57 and 36 < lon < 39:
                    return [lat, lon]
                
            return None
        except Exception as e:
            return None
    
    # 4. Обработка данных
    heat_data = []
    marker_cluster = MarkerCluster().add_to(premises_group)
    added_count = 0
    
    for idx, row in premises.iterrows():
        coords = extract_coords_from_csv(row['geoData'])
        if coords and pd.notna(row['Area']) and row['Area'] > 0:
            heat_data.append(coords)
            added_count += 1
            
            # Добавляем маркер
            folium.Marker(
                coords,
                popup=f"""
                    <b>Адрес:</b> {row['Адрес']}<br>
                    <b>Площадь:</b> {row['Area']:.1f} кв.м<br>
                    <b>Район:</b> {row.get('Район', 'не указан')}
                """,
                icon=folium.Icon(icon='building', color='blue')
            ).add_to(marker_cluster)
    
    # 5. Создаем тепловую карту
    if heat_data:
        HeatMap(
            heat_data,
            radius=15,
            blur=20,
            min_opacity=0.5,
            gradient={0.4: 'blue', 0.6: 'lime', 1: 'red'}
        ).add_to(heatmap_group)
        print(f"Успешно обработано {added_count} помещений из {len(premises)}")
    else:
        print("Нет данных для тепловой карты")
        folium.Marker(
            [55.76, 37.61],
            popup='Нет данных по помещениям',
            icon=folium.Icon(color='gray', icon='info-sign')
        ).add_to(premises_group)

except Exception as e:
    print(f"Ошибка создания слоя помещений: {e}")
    import traceback
    traceback.print_exc()
    folium.Marker(
        [55.75, 37.62],
        popup=f'Ошибка загрузки помещений: {e}',
        icon=folium.Icon(color='red', icon='alert')
    ).add_to(premises_group)

# Добавляем слои на карту
premises_group.add_to(m)
heatmap_group.add_to(m)

print("Слои помещений добавлены на карту")

Успешно обработано 55453 помещений из 55458
Слои помещений добавлены на карту


Создаем 4 слой (земельные участки)

In [7]:
try:
    # 1. Загрузка данных с проверкой столбцов
    land_plot = pd.read_csv(
        "land_plot.csv",
        sep=';',
        encoding='utf-8',
        skiprows=1
    )
    
    # Проверяем наличие необходимых столбцов
    required_columns = ['Кадастровый номер земельного участка', 'Площадь земельного участка (кв.м.)', 'geoData']
    for col in required_columns:
        if col not in land_plot.columns:
            raise ValueError(f"Отсутствует обязательный столбец: {col}")
    
    # 2. Создаем идентификатор, если нет столбца ID
    if 'ID' not in land_plot.columns:
        land_plot['ID'] = land_plot['Кадастровый номер земельного участка']
    
    # 3. Обработка площади
    def safe_convert(area):
        try:
            return float(str(area).replace(',', '.'))
        except:
            return 0.0
    
    land_plot['Area'] = land_plot['Площадь земельного участка (кв.м.)'].apply(safe_convert)
    
    # 4. Универсальный парсинг координат
    def extract_coords(geo_str):
        try:
            if pd.isna(geo_str):
                return None
                
            # Преобразуем в строку
            geo_str = str(geo_str)
            
            # Ищем все пары координат
            coord_pairs = re.findall(r'(\d+\.\d+)[,\s]+(\d+\.\d+)', geo_str)
            if len(coord_pairs) < 3:
                return None
                
            # Преобразуем в (lon, lat)
            coords = []
            for pair in coord_pairs:
                try:
                    lon = float(pair[0])
                    lat = float(pair[1])
                    coords.append((lon, lat))
                except:
                    continue
            
            if len(coords) < 3:
                return None
                
            # Вычисляем центр
            lats = [p[1] for p in coords]
            lons = [p[0] for p in coords]
            center = [sum(lats)/len(lats), sum(lons)/len(lons)]
            
            return {
                'center': center,
                'coords': coords
            }
        except Exception as e:
            print(f"Ошибка обработки координат: {e}")
            return None
    
    # 5. Обработка данных
    valid_plots = []
    for idx, row in land_plot.iterrows():
        coords = extract_coords(row['geoData'])
        if coords:
            valid_plots.append({
                'id': row['ID'],
                'cadastral': row['Кадастровый номер земельного участка'],
                'area': row['Area'],
                'center': coords['center'],
                'point_count': len(coords['coords'])
            })
    
    if not valid_plots:
        raise ValueError("Не найдено участков с валидными координатами")
    
    # 6. Создаем кластер маркеров
    marker_cluster = MarkerCluster().add_to(land_group)
    
    # 7. Добавляем маркеры
    for plot in valid_plots:
        folium.Marker(
            plot['center'],
            popup=f"""
                <b>Кадастр:</b> {plot['cadastral']}<br>
                <b>Площадь:</b> {plot['area']:.1f} кв.м<br>
                <b>Точек границы:</b> {plot['point_count']}
            """,
            icon=folium.Icon(icon='tree', color='green')
        ).add_to(marker_cluster)
    
    print(f"Успешно добавлено {len(valid_plots)} участков")

except Exception as e:
    print(f"Ошибка: {e}")
    folium.Marker(
        [55.75, 37.61],
        popup=f'Ошибка загрузки участков: {e}',
        icon=folium.Icon(color='red', icon='alert')
    ).add_to(land_group)

# Добавляем слой на карту
land_group.add_to(m)

print("Слой земельных участков добавлен на карту")

Успешно добавлено 100961 участков
Слой земельных участков добавлен на карту


Дополнительные действия

In [ ]:
# Добавляем LayerControl для управления слоями
layer_control = folium.LayerControl(
    position='topright',
    collapsed=False,
    autoZIndex=True
).add_to(m)

# Сохраняем карту
m.save('final_map.html')
print("Карта сохранена как final_map.html")
print("Всего слоев на карте: 5")
print("- Объекты приватизации")
print("- Объекты аренды")
print("- Нежилые помещения")
print("- Плотность нежилых помещений")
print("- Земельные участки")

Сохраняем карту